In [22]:
import torch
from skimage import io
import onnxruntime as ort
import onnx
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [23]:
model = onnx.load('HPA_FOV_data/densenet_model.onnx')


In [24]:
from onnx2torch import convert


In [25]:
try:
    onnx.checker.check_model('HPA_FOV_data/densenet_model_batch.onnx')
except onnx.checker.ValidationError as e:
    print(f"The model is invalid: {e}")
else:
    print("The model is valid!")


The model is valid!


In [27]:
ort_session = ort.InferenceSession("HPA_FOV_data/densenet_model.onnx", None, providers=["CUDAExecutionProvider"])

In [16]:
all_features = torch.zeros(len(dataset), 1024).float()
all_proteins = torch.zeros(len(dataset), 28).int()


In [25]:
ort_session.get_inputs()


In [11]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
from utils import Multilabel_classifier
import torch
import vision_transformer as vits
from file_dataset import ImageFileList, AutoBalancedFileList, default_loader, pandas_reader, pandas_reader_no_labels, pandas_reader_binary_labels
from tqdm import tqdm
from yaml_tfms import tfms_from_config
import numpy as np
from label_dict import protein_to_num_full as protein_to_num
from sklearn.metrics import f1_score
import yaml
import pandas as pd
from pathlib import Path
import torch
import importlib
import file_dataset
from label_dict import protein_to_num_full
importlib.reload(file_dataset)
ImageFileList = file_dataset.ImageFileList
pandas_reader_binary_labels = file_dataset.pandas_reader_binary_labels
from tqdm import tqdm
import argparse
import torch
import importlib
import utils
importlib.reload(utils)
from utils import Multilabel_classifier, simple_clf, expanding_clf, prototyping_clf, residual_clf, residual_add_clf
import protein_dataset
importlib.reload(protein_dataset)
from protein_dataset import ProteinDataset 

batch_size_per_gpu = 1

dataset = ProteinDataset(
        '/scr/mdoron/Dino4Cells/data/kaggle_whole_image_test_IDs.csv',
        img_size=1536,
        is_trainset=False,
        return_label=False,
        in_channels=4,
        transform=None,
        crop_size=1024,
        random_crop=False,
        target_labels=sorted(list(protein_to_num_full.keys()))
)

kaggle_data_loader = torch.utils.data.DataLoader(
    dataset,
#     sampler=train_sampler,
    batch_size=batch_size_per_gpu,
    num_workers=10,
    pin_memory=True,
    drop_last=False,
)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
IDs = []
features = []
cell_lines = []
proteins = []
bad_IDs = []
outputs = []
all_features = torch.zeros(len(dataset), 1024).float()
all_proteins = torch.zeros(len(dataset), 28).float()
i = 0
for img, ID in tqdm(kaggle_data_loader):
# for img, protein, cell, ID in tqdm(kaggle_data_loader):
    img1 = img
    try:
        ort_outs = ort_session.run(None, {'image': to_numpy(img)})
        IDs.extend(ID)
        outputs.append(ort_outs[0])
    except:
        bad_IDs.append(ID)

# torch.save((all_features, all_proteins, cell_lines, IDs), 'bestfitting_test_features.pth')        



/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/test/images_1536
/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/train/external_v18_1536


  1%|█                                                                                                                                                                   | 75/11702 [00:13<33:51,  5.72it/s]


ValueError: too many values to unpack (expected 2)